In [2]:
import os
import cv2
import numpy as np
import torch

In [3]:
# Specify the folder containing the images and the output folder
input_folder = '../datasets/thermal/images/train2017/'
output_folder_t = '../datasets/t-yolov5/images'
output_folder_t2 = '../datasets/t2-yolov5/images'
image_files = os.listdir(input_folder)
    
# Sort the image files
#image_files.sort()

# Iterate over each image file
for i, image_file in enumerate(image_files):
    if image_file.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        # Read the current image
        current_image_path = os.path.join(input_folder, image_file)
        current_image = cv2.resize(cv2.imread(current_image_path, cv2.IMREAD_GRAYSCALE),(640,640))
        # Get the file names of the previous and next images
        prev_image_file = image_files[i-1] if i > 0 else image_files[i]
        next_image_file = image_files[i+1] if i < len(image_files)-1 else image_files[i]
        # Read the previous and next images
        prev_image = cv2.resize((cv2.imread(os.path.join(input_folder, prev_image_file), cv2.IMREAD_GRAYSCALE) if prev_image_file else None),(640,640))
        next_image = cv2.resize((cv2.imread(os.path.join(input_folder, next_image_file), cv2.IMREAD_GRAYSCALE) if next_image_file else None),(640,640))
        
        first_channel = np.abs(prev_image-current_image)
        second_channel= np.abs(next_image-current_image)

        t_image = np.dstack([current_image,prev_image,next_image])
        t2_image = np.dstack([first_channel,second_channel,np.zeros_like(first_channel)])
        t2_image = np.concatenate([t_image,t2_image],axis=1)
        # Save the transformed image
        output_path_t = os.path.join(output_folder_t, image_file)
        output_path_t2 = os.path.join(output_folder_t2, image_file)
        cv2.imwrite(output_path_t, t_image)
        cv2.imwrite(output_path_t2,t2_image)



In [ ]:
!python train.py --img 640 --epochs 300 --data t-yolov5.yaml --weights yolov5x.pt --name t-yolov5

In [38]:
!python train.py --img 640 --epochs 300 --data t2-yolov5.yaml --cfg yolov5x-t2.yaml --name t2-yolov5 --t2_yolov5 True

11895.23s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


train: weights=yolov5s.pt, cfg=yolov5x-t2.yaml, data=t2-yolov5.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=t2-yolov5, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, t2_yolov5=True, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-196-gf5eb864 Python-3.10.9 torch-2.0.0 CUDA:0 (NVIDIA GeForce GTX 1650, 3904MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, f